In [1]:
from utils_bern import *
import numpy as np
from scipy.stats import bernoulli
import scipy.stats as ss
import pymc3.stats as pymcs
import pickle
import argparse
from easydict import EasyDict as edict
np.random.seed(0)

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
results = []
p0 = 0.4
n = 120
ps = [0.2, 0.4]
ns = [50, 100]
cutoff = 0.025

In [4]:
result = edict()
D = bernoulli.rvs(p0, size=n)
Ds = [ bernoulli.rvs(ps[i], size=ns[i]) for i in range(len(ns))] 
result["data"] = {"D":D, 
                  "Ds": Ds, 
                  "p0": p0,
                  "ps": ps,
                  "n" : n ,
                  "ns": ns
                  } 

### UIP-JS-Bernoulli

In [12]:
UIPJS_model = getUIPJSBern(D, Ds)
with UIPJS_model:
    step = pm.Metropolis()
    post_Bern_UIPJS = pm.sample(draws=5000, tune=5000, 
                                #target_accept=0.9, 
                                step=step,
                                cores=4, chains=4)
result["UIPJS"] = post_Bern_UIPJS

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [thetah]
>Metropolis: [M]
Sampling 4 chains, 0 divergences: 100%|██████████| 40000/40000 [00:03<00:00, 10602.04draws/s]
The number of effective samples is smaller than 25% for some parameters.


In [13]:
pm.summary(post_Bern_UIPJS)

,mean,sd,hpd_3%,hpd_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
M,75.944,41.842,13.583,149.998,0.751,0.535,3105.0,3060.0,2988.0,3232.0,1.0
thetah,0.369,0.038,0.298,0.441,0.001,0.000,2980.0,2971.0,2987.0,3770.0,1.0


### UIP-D-Bernoulli

In [14]:
UIPD_model = getUIPDBern(D, Ds)
with UIPD_model:
    step = pm.Metropolis()
    post_Bern_UIPD = pm.sample(draws=5000, tune=5000, 
                               #target_accept=0.9, 
                               step=step,
                               cores=4, chains=4)
result["UIPD"] = post_Bern_UIPD

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [thetah]
>Metropolis: [M]
>Metropolis: [pis]
Sampling 4 chains, 0 divergences: 100%|██████████| 40000/40000 [00:04<00:00, 8011.30draws/s]
The number of effective samples is smaller than 25% for some parameters.


In [15]:
pm.summary(post_Bern_UIPD)

,mean,sd,hpd_3%,hpd_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
pis[0],0.194,0.236,0.000,0.688,0.004,0.003,3943.0,3943.0,3307.0,3804.0,1.0
pis[1],0.806,0.236,0.312,1.000,0.004,0.003,3943.0,3810.0,3307.0,3804.0,1.0
M,74.321,41.234,5.307,141.285,0.684,0.484,3636.0,3636.0,3656.0,4392.0,1.0
thetah,0.370,0.037,0.301,0.439,0.001,0.000,3147.0,3147.0,3136.0,4057.0,1.0


### NPP-Bernoulli

In [16]:
NPP_model = getNPPBern(D, Ds)
with NPP_model:
    step = pm.Metropolis()
    post_Bern_NPP = pm.sample(draws=5000, tune=5000, 
                              #target_accept=0.8, 
                              step=step,
                              cores=4, chains=4)
result["NPP"] = post_Bern_NPP

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [thetah]
>Metropolis: [gammas]
Sampling 4 chains, 0 divergences: 100%|██████████| 40000/40000 [00:03<00:00, 10918.87draws/s]
The number of effective samples is smaller than 25% for some parameters.


In [17]:
pm.summary(post_Bern_NPP)

,mean,sd,hpd_3%,hpd_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
gammas[0],0.418,0.280,0.000,0.899,0.006,0.004,2217.0,2217.0,2027.0,2628.0,1.0
gammas[1],0.536,0.277,0.086,0.998,0.005,0.004,2537.0,2537.0,2294.0,3025.0,1.0
thetah,0.367,0.036,0.304,0.438,0.001,0.000,3109.0,3098.0,3100.0,4307.0,1.0


In [20]:
np.quantile(post_Bern_NPP["thetah"], [0.025, 0.5, 0.975])

array([0.29833591, 0.36572194, 0.43917414])

### rMAP-Bernoulli

In [11]:
rMAPprior = getMixBeta(Ds)
rMAPpost = MixPostBeta(rMAPprior, D)
post_Bern_rMAP = genMixBeta(20000, rMAPpost)
result["rMAP"] = post_Bern_rMAP

R[write to console]: Loading required package: Rcpp

R[write to console]: This is RBesT version 1.6.1

R[write to console]: Assuming default prior dispersion for beta: 2

R[write to console]: Assuming default prior location   for beta: 0



In [18]:
np.quantile(post_Bern_rMAP, [0.025, 0.5, 0.975])

array([0.30245117, 0.37864944, 0.47111509])

In [19]:
np.mean(post_Bern_rMAP)

0.3807579844980737